# import function

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from __future__ import print_function
import random
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, BatchNormalization, Input
from keras.optimizers import Adam
from keras import callbacks
from keras import backend as K

from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten

# Load dataset

In [ ]:
def get_data(mins):
    mins = str(mins)
    with open ('data_list_' + mins, 'rb') as fp:
        data_list = np.array(pickle.load(fp))
        #data_list = np.reshape(data_list, (data_list.shape[0], data_list.shape[1], 1, data_list.shape[2]))
        data_list = np.reshape(data_list, (data_list.shape[0], data_list.shape[1], data_list.shape[2], 1))
    with open ('label_list_' + mins, 'rb') as fp:
        label_list = np.array(pickle.load(fp))

    print(data_list.shape)
    print(label_list.shape)
    
    return data_list, label_list

# Recall function

In [ ]:
def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    
    return recall

# Precision function

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.cast(K.greater(K.clip(y_true * y_pred, 0, 1), 0.20), 'float32'))
    pred_positives = K.sum(K.cast(K.greater(K.clip(y_pred, 0, 1), 0.20), 'float32'))

    precision = true_positives / (pred_positives + K.epsilon())
    return precision

# F-measure function

In [ ]:
def f_measure(y_true, y_pred):
    p_val = precision(y_true, y_pred)
    r_val = recall(y_true, y_pred)
    f_val = 2*p_val*r_val / (p_val + r_val)

    return f_val

# Building module

In [ ]:
def create_model():
    model = Sequential()
    model.add(Convolution2D(batch_input_shape=(None, 10, 38, 1),
                            filters=32,
                            kernel_size=(3, 3),
                            strides=1,
                            padding='valid',
                            data_format='channels_last',
                            activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=2,
                           padding='valid',
                           data_format='channels_last'))
    
    ##model.add(Dropout(0.7))
    model.add(Convolution2D(filters=64,
                            kernel_size=(3, 3),
                            strides=1,
                            padding='valid',
                            data_format='channels_last',
                            activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=3,
                           padding='valid',
                           data_format='channels_last'))
    
    #model.add(Dropout(0.7))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Dense(4,
                    activation='sigmoid'))

    adam = Adam(0.001)
    model.compile(optimizer=adam,
                  loss='binary_crossentropy',
                  metrics=['acc', precision, recall, f_measure])

    model.summary()
    return model

## Load dataset and Spliting training set and validation set

In [ ]:
history_list = []
train_size_rate = 0.9
BATCH_SIZE = 32
EPOCHS = 200

model = create_model()
data_list, label_list = get_data('10_10')

In [ ]:
train_data, test_data, train_label, test_label = train_test_split(data_list, label_list, train_size=train_size_rate)

# Start training model

In [ ]:
model.fit(train_data, train_label,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(test_data, test_label),
                    shuffle=True)

# Test testing model

In [ ]:
test_data_list, test_label_list = get_data('10_10_test')
model.evaluate(test_data_list, test_label_list, batch_size=32, verbose=1, sample_weight=None)

# Draw ROC

In [ ]:
import matplotlib

def plot_filters(layer, x, y):
    filters = model.layers[layer].get_weights()[0][:,:,:,:]
    fig = plt.figure()
    for j in range(len(filters)):
        ax = fig.add_subplot(y, x, j+1)
        ax.matshow(filters[j][0], cmap = matplotlib.cm.binary)
        plt.xticks(np.arrary([]))
        plt.yticks(np.arrary([]))
    plt.tight_layout()
    return plt

In [ ]:
from sklearn.metrics import roc_curve
y_pred = model.predict_proba(test_data).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_label.ravel(), y_pred)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(8,8))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Area = {:.3f}'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()